In [1]:
import numpy as np
import SimpleITK as sitk
import skimage
import os, csv
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


In [2]:
progress_path = '/data/scratch/aalexopoulos/OAI_IW-TSE_progress_flipped/'
progress_imgs = sorted(os.path.join(progress_path, fname) for fname in os.listdir(progress_path))

progress_mask_path = '/data/scratch/aalexopoulos/OAI_IW-TSE_progress_seg_flipped/'
progress_mask = sorted(os.path.join(progress_mask_path, fname) for fname in os.listdir(progress_mask_path))

In [3]:
# FUNCTION THAT APPLIES CANNY EDGE FILTER IN EACH SLICE OF THE INPUT MASK 
# AND RETURNS THE EDGE MATRICES OF 0 AND 1 PIXEL VALUES

def edges_calc(patient_mask_np):
    edges_all = []
    edges_fb = []
    edges_tb = []
    for i in range(patient_mask_np.shape[0]):
        # The threshold is set to 500 bone pixels in order to minimize the misclassified bone pixels 
        # present in the mask that will be used for canny edge detection filters 
        # and thus in the search for min and max coords of tibia and femur bones
        if len(np.where(patient_mask_np[i,:,:]>0.0)[0])>500:
            edges1 = skimage.feature.canny(patient_mask_np[i,:,:],sigma=8,low_threshold=0.05,high_threshold=0.4)
            edges_all.append(edges1)

            patient_mask_np_fb = patient_mask_np[i,:,:].copy() 
            patient_mask_np_fb[np.where(patient_mask_np_fb!=1.0)]=0.0
            edges_2 = skimage.feature.canny(patient_mask_np_fb,sigma=8,low_threshold=0.37,high_threshold=0.39)
            edges_fb.append(edges_2)

            patient_mask_np_tb = patient_mask_np[i,:,:].copy() 
            patient_mask_np_tb[np.where(patient_mask_np_tb!=3.0)]=0.0
            edges_3 = skimage.feature.canny(patient_mask_np_tb,sigma=8,low_threshold=0.9,high_threshold=0.95)
            edges_tb.append(edges_3)
    
    edges_all_np = np.array(edges_all).astype(int)
    edges_fb_np = np.array(edges_fb).astype(int)
    edges_tb_np = np.array(edges_tb).astype(int)

    return edges_all_np, edges_fb_np, edges_tb_np



In [52]:
# FUNCTION THAT CALCULATES THE MINIMUM AND MAXIMUM X AND Y COORDINATES
# FOR THE FEMURAL AND TIBIAL BONES FOR EACH PATIENT

# receives as input femur_edges(,384,384) and tibia_edges(,384,384)
# and returns as output the X,Y coords of Xmin, Xmax for both Femor and Tibia Bones
# and the X,Y coords of Ymin of the Tibial Bone

# Here the thresholds are held in ZERO for the number of ones pixel values for the Canny Edge Images
# because the minimization of misclassified bone pixels was applied in the Canny Edge calculation function

def max_min_coords(femur_edges, tibia_edges):
    #firstly find min and max X coords of FEMUR for each slice mask
    ind_fb_xmin = []
    ind_fb_xmax = []
    for i in range(femur_edges.shape[0]):
        if len(np.where(femur_edges[i,:,:]==1.0)[0])>0:
            indices_1 = np.where(femur_edges[i,:,:]==1.0)
            indices_1 = np.array(indices_1)
            sorted_indices_1 = sorted((tp1 for tp1 in zip(*indices_1)), key=lambda x:x[1])
            ind_fb_xmin.append(sorted_indices_1[0])
            ind_fb_xmax.append(sorted_indices_1[-1])
    
    #secondly find min X coords of FEMUR for one input patient mask
    ind_fb_xmin_np = np.array(ind_fb_xmin)
    if ind_fb_xmin_np.ndim>1:    
        ind_fb_xmin_global = ind_fb_xmin_np[np.where(ind_fb_xmin_np[:,1]==np.amin(ind_fb_xmin_np[:,1])),:]
        if ind_fb_xmin_global.shape[1] == 1:
            fb_xmin_global_xcoord = ind_fb_xmin_global[0][0][1]
            fb_xmin_global_ycoord = ind_fb_xmin_global[0][0][0]
        if ind_fb_xmin_global.shape[1] > 1:
            idx = np.where(ind_fb_xmin_global[:,:,0] == np.min(ind_fb_xmin_global[:,:,0]))
            fb_xmin_global_xcoord = ind_fb_xmin_global[idx][0][1]
            fb_xmin_global_ycoord = ind_fb_xmin_global[idx][0][0]
    xmin_femur_coords = np.array([fb_xmin_global_xcoord,fb_xmin_global_ycoord])

    #thirdly find max X coords of FEMUR for one input patient mask
    ind_fb_xmax_np = np.array(ind_fb_xmax)
    if ind_fb_xmax_np.ndim>1:
        ind_fb_xmax_global = ind_fb_xmax_np[np.where(ind_fb_xmax_np[:,1]==np.amax(ind_fb_xmax_np[:,1])),:]
        if ind_fb_xmax_global.shape[1] == 1:
            fb_xmax_global_xcoord = ind_fb_xmax_global[0][0][1]
            fb_xmax_global_ycoord = ind_fb_xmax_global[0][0][0]
        if ind_fb_xmax_global.shape[1] > 1:
            idx = np.where(ind_fb_xmax_global[:,:,0] == np.max(ind_fb_xmax_global[:,:,0]))
            fb_xmax_global_xcoord = ind_fb_xmax_global[idx][0][1]
            fb_xmax_global_ycoord = ind_fb_xmax_global[idx][0][0]
    xmax_femur_coords = np.array([fb_xmax_global_xcoord, fb_xmax_global_ycoord])

    # NOW THE SAME FOR TIBIAL BONE
    # Firstly find all min and max values for X coord of TIBIA for each slice
    ind_tb_xmin = []
    ind_tb_xmax = []
    ind_tb_ymin = []
    for i in range(tibia_edges.shape[0]):
        if len(np.where(tibia_edges[i,:,:]==1.0)[0])>0:
            indices_1 = np.where(tibia_edges[i,:,:]==1.0)
            indices_1 = np.array(indices_1)
            sorted_indices_1 = sorted((tp1 for tp1 in zip(*indices_1)), key=lambda x:x[1])
            ind_tb_xmin.append(sorted_indices_1[0])
            ind_tb_xmax.append(sorted_indices_1[-1])
            sorted_indices_2 = sorted((tp2 for tp2 in zip(*indices_1)), key=lambda x:x[0])
            ind_tb_ymin.append(sorted_indices_2[0])
    
    # Secondly find min values for X coord of TIBIA for one input patient mask
    ind_tb_xmin_np = np.array(ind_tb_xmin)
    if ind_tb_xmin_np.ndim>1:
        ind_tb_xmin_global = ind_tb_xmin_np[np.where(ind_tb_xmin_np[:,1]==np.amin(ind_tb_xmin_np[:,1])),:]
        if ind_tb_xmin_global.shape[1]==1:    
            tb_xmin_global_xcoord = ind_tb_xmin_global[0][0][1]
            tb_xmin_global_ycoord = ind_tb_xmin_global[0][0][0]
        if ind_tb_xmin_global.shape[1]>1:
            idx = np.where(ind_tb_xmin_global[:,:,0]==np.min(ind_tb_xmin_global[:,:,0]))
            tb_xmin_global_xcoord = ind_tb_xmin_global[idx][0][1]
            tb_xmin_global_ycoord = ind_tb_xmin_global[idx][0][0]
    xmin_tibia_coords = np.array([tb_xmin_global_xcoord,tb_xmin_global_ycoord])

    # Thirdly find max values for X coord of TIBIA for one input patient mask
    ind_tb_xmax_np = np.array(ind_tb_xmax)
    if ind_tb_xmax_np.ndim>1:
        ind_tb_xmax_global = ind_tb_xmax_np[np.where(ind_tb_xmax_np[:,1]==np.amax(ind_tb_xmax_np[:,1])),:]
        if ind_tb_xmax_global.shape[1]==1 :
            tb_xmax_global_xcoord = ind_tb_xmax_global[0][0][1]
            tb_xmax_global_ycoord = ind_tb_xmax_global[0][0][0]
        if ind_tb_xmax_global.shape[1]>1 :
            idx = np.where(ind_tb_xmax_global[:,:,0]==np.max(ind_tb_xmax_global[:,:,0]))
            tb_xmax_global_xcoord = ind_tb_xmax_global[idx][0][1]
            tb_xmax_global_ycoord = ind_tb_xmax_global[idx][0][0]
    xmax_tibia_coords = np.array([tb_xmax_global_xcoord,tb_xmax_global_ycoord])

    # Fourthly find min values for Y coord of TIBIA for one input patient mask
    ind_tb_ymin_np = np.array(ind_tb_ymin)
    if ind_tb_ymin_np.ndim>1:
        ind_tb_ymin_global = ind_tb_ymin_np[np.where(ind_tb_ymin_np[:,0]==np.amin(ind_tb_ymin_np[:,0])),:]
        if ind_tb_ymin_global.shape[1]==1:    
            tb_ymin_global_xcoord = ind_tb_ymin_global[0][0][1]
            tb_ymin_global_ycoord = ind_tb_ymin_global[0][0][0]
        if ind_tb_ymin_global.shape[1]>1:
            idx = np.where(ind_tb_ymin_global[:,:,1]==np.amin(ind_tb_ymin_global[:,:,1]))
            tb_ymin_global_xcoord = ind_tb_ymin_global[idx][0][1]
            tb_ymin_global_ycoord = ind_tb_ymin_global[idx][0][0]
    ymin_tibia_coords = np.array([tb_ymin_global_xcoord,tb_ymin_global_ycoord])

    return xmin_femur_coords, xmax_femur_coords, xmin_tibia_coords, xmax_tibia_coords, ymin_tibia_coords

In [69]:
# HERE WE CREATE THE LISTS THAT WILL HOLD THE COORDINDATES OF MIN AND MAX TIBIAL AND FEMURAL VALUES
progress_femural_edges_50 = []
progress_tibial_edges_50 = []

progress_fb_xmin_c_50 = []
progress_fb_xmax_c_50 = []
progress_tb_xmin_c_50 = []
progress_tb_xmax_c_50 = []
progress_tb_ymin_c_50 = []

# bla = ['p_num','FB Xmin Xcoord','FB Xmin Ycoord','FB Xmax Xcoord', 'FB Xmax Ycoord',
        # 'TB Xmin Xcoord','TB Xmin Ycoord','TB Xmax Xcoord', 'TB Xmax Ycoord','TB Ymin Xcoord','TB Ymin Ycoord']

progress_all_coords_50 = np.empty((0,11),dtype=int)


In [62]:
k = 0

for i in range(0,len(progress_imgs[:50])):
    p_num = progress_imgs[i].split('/')[-1]
    mask = sitk.ReadImage(progress_mask[i])
    mask_np = sitk.GetArrayFromImage(mask)
    
    image = sitk.ReadImage(progress_imgs[i])
    image_np = sitk.GetArrayFromImage(image)

    knee_edges, fb_edges, tb_edges = edges_calc(mask_np)
    progress_femural_edges_50.append(fb_edges)
    progress_tibial_edges_50.append(tb_edges)

    fb_xmin_c, fb_xmax_c, tb_xmin_c, tb_xmax_c, tb_ymin_c = max_min_coords(fb_edges, tb_edges)
    
    progress_fb_xmin_c_50.append(fb_xmin_c)
    progress_fb_xmax_c_50.append(fb_xmax_c)
    progress_tb_xmin_c_50.append(tb_xmin_c)
    progress_tb_xmax_c_50.append(tb_xmax_c)
    progress_tb_ymin_c_50.append(tb_ymin_c)
    all_info = np.array([[k,fb_xmin_c[0],fb_xmin_c[1],fb_xmax_c[0],fb_xmax_c[1],tb_xmin_c[0],tb_xmin_c[1],tb_xmax_c[0],tb_xmax_c[1],tb_ymin_c[0],tb_ymin_c[1]]])
    progress_all_coords_50 = np.append(progress_all_coords_50,all_info,axis=0)
    
    print(progress_imgs[i].split('/')[-1],k)
    k = k +1


9001400_IW-TSE_R.nii.gz 0
9003380_IW-TSE_R.nii.gz 1
9004175_IW-TSE_R.nii.gz 2
9004184_IW-TSE_R.nii.gz 3
9004669_IW-TSE_L.nii.gz 4
9009927_IW-TSE_L.nii.gz 5
9009957_IW-TSE_L.nii.gz 6
9015798_IW-TSE_R.nii.gz 7
9021102_IW-TSE_L.nii.gz 8
9022789_IW-TSE_R.nii.gz 9
9025191_IW-TSE_R.nii.gz 10
9025994_IW-TSE_R.nii.gz 11
9029791_IW-TSE_R.nii.gz 12
9029913_IW-TSE_R.nii.gz 13
9031141_IW-TSE_R.nii.gz 14
9033937_IW-TSE_R.nii.gz 15
9034644_IW-TSE_R.nii.gz 16
9034812_IW-TSE_R.nii.gz 17
9036287_IW-TSE_L.nii.gz 18
9038876_IW-TSE_R.nii.gz 19
9039627_IW-TSE_R.nii.gz 20
9040944_IW-TSE_L.nii.gz 21
9048979_IW-TSE_L.nii.gz 22
9048979_IW-TSE_R.nii.gz 23
9049447_IW-TSE_R.nii.gz 24
9049895_IW-TSE_L.nii.gz 25
9050299_IW-TSE_L.nii.gz 26
9051960_IW-TSE_L.nii.gz 27
9055361_IW-TSE_R.nii.gz 28
9058469_IW-TSE_R.nii.gz 29
9062645_IW-TSE_L.nii.gz 30
9064631_IW-TSE_R.nii.gz 31
9065433_IW-TSE_R.nii.gz 32
9066155_IW-TSE_L.nii.gz 33
9070903_IW-TSE_R.nii.gz 34
9071924_IW-TSE_L.nii.gz 35
9072956_IW-TSE_L.nii.gz 36
9081635_IW-

In [63]:
progress_fb_xmax_c_50_np = np.array(progress_fb_xmax_c_50)
progress_fb_xmin_c_50_np = np.array(progress_fb_xmin_c_50)
progress_tb_xmax_c_50_np = np.array(progress_tb_xmax_c_50)
progress_tb_xmin_c_50_np = np.array(progress_tb_xmin_c_50)
progress_tb_ymin_c_50_np = np.array(progress_tb_ymin_c_50)

In [65]:
save_path_50 = '/data/scratch/aalexopoulos/coords_outputs/all_progress_50_coords.csv'
np.savetxt(save_path_50,progress_all_coords_50,delimiter=',')


In [8]:
fb_xmax_c_global = progress_fb_xmax_c_50_np[np.where(progress_fb_xmax_c_50_np[:,0] == np.amax(progress_fb_xmax_c_50_np[:,0]))]
fb_xmax_c_global_xcoord = fb_xmax_c_global[0][0]
fb_xmax_c_global_ycoord = fb_xmax_c_global[0][1]
print('Global progress femural xmax x coord: {}, y coord: {}'.format(fb_xmax_c_global_xcoord, fb_xmax_c_global_ycoord))
print(progress_imgs[int(np.where(progress_fb_xmax_c_50_np[:,0] == np.amax(progress_fb_xmax_c_50_np[:,0]))[0])].split('/')[-1])

fb_xmin_c_global = progress_fb_xmin_c_50_np[np.where(progress_fb_xmin_c_50_np[:,0] == np.amin(progress_fb_xmin_c_50_np[:,0]))]
fb_xmin_c_global_xcoord = fb_xmin_c_global[0][0]
fb_xmin_c_global_ycoord = fb_xmin_c_global[0][1]
print('Global progress femural xmin x coord: {}, y coord: {}'.format(fb_xmin_c_global_xcoord, fb_xmin_c_global_ycoord))
print(progress_imgs[int(np.where(progress_fb_xmin_c_50_np[:,0] == np.amin(progress_fb_xmin_c_50_np[:,0]))[0])].split('/')[-1])

tb_xmax_c_global = progress_tb_xmax_c_50_np[np.where(progress_tb_xmax_c_50_np[:,0] == np.amax(progress_tb_xmax_c_50_np[:,0]))]
tb_xmax_c_global_xcoord = tb_xmax_c_global[0][0]
tb_xmax_c_global_ycoord = tb_xmax_c_global[0][1]
print('Global progress tibial xmax x coord: {}, y coord: {}'.format(tb_xmax_c_global_xcoord, tb_xmax_c_global_ycoord))
print(progress_imgs[int(np.where(progress_tb_xmax_c_50_np[:,0] == np.amax(progress_tb_xmax_c_50_np[:,0]))[0])].split('/')[-1])

tb_xmin_c_global = progress_tb_xmin_c_50_np[np.where(progress_tb_xmin_c_50_np[:,0] == np.amin(progress_tb_xmin_c_50_np[:,0]))]
tb_xmin_c_global_xcoord = tb_xmin_c_global[0][0]
tb_xmin_c_global_ycoord = tb_xmin_c_global[0][1]
print('Global progress tibial xmin x coord: {}, y coord: {}'.format(tb_xmin_c_global_xcoord, tb_xmin_c_global_ycoord))
print(progress_imgs[int(np.where(progress_tb_xmin_c_50_np[:,0] == np.amin(progress_tb_xmin_c_50_np[:,0]))[0])].split('/')[-1])

tb_ymin_c_global = progress_tb_ymin_c_50_np[np.where(progress_tb_ymin_c_50_np[:,1] == np.amin(progress_tb_ymin_c_50_np[:,1]))]
tb_ymin_c_global_xcoord = tb_ymin_c_global[0][0]
tb_ymin_c_global_ycoord = tb_ymin_c_global[0][1]
print('Global progress tibial ymin x coord: {}, y coord: {}'.format(tb_ymin_c_global_xcoord, tb_ymin_c_global_ycoord))
print(progress_imgs[int(np.where(progress_tb_ymin_c_50_np[:,1] == np.amin(progress_tb_ymin_c_50_np[:,1]))[0])].split('/')[-1])
tb_ymin_c_global, tb_xmin_c_global, tb_xmax_c_global, fb_xmin_c_global, fb_xmax_c_global

# bla = ['p_num','FB Xmin Xcoord','FB Xmin Ycoord','FB Xmax Xcoord', 'FB Xmax Ycoord',
        # 'TB Xmin Xcoord','TB Xmin Ycoord','TB Xmax Xcoord', 'TB Xmax Ycoord','TB Ymin Xcoord','TB Ymin Ycoord']

Global control femural xmax x coord: 319, y coord: 149
9034812_IW-TSE_R.nii.gz
Global control femural xmin x coord: 33, y coord: 135
9004669_IW-TSE_L.nii.gz
Global control tibial xmax x coord: 303, y coord: 271
9129270_IW-TSE_R.nii.gz
Global control tibial xmin x coord: 43, y coord: 250
9004669_IW-TSE_L.nii.gz
Global control tibial ymin x coord: 172, y coord: 179
9100442_IW-TSE_R.nii.gz


(array([[172, 179]]),
 array([[ 43, 250]]),
 array([[303, 271]]),
 array([[ 33, 135]]),
 array([[319, 149]]))

In [9]:
global_progress_coords = {'Global progress FB xmax x coord':fb_xmax_c_global_xcoord,'Global progress FB xmax y coord':fb_xmax_c_global_ycoord,
                        'Global progress FB xmin x coord':fb_xmin_c_global_xcoord,'Global progress FB xmin y coord':fb_xmin_c_global_ycoord,
                        'Global progress TB xmax x coord':tb_xmax_c_global_xcoord,'Global progress tb xmax y coord':tb_xmax_c_global_ycoord,
                        'Global progress tb xmin x coord':tb_xmin_c_global_xcoord,'Global progress tb xmin y coord':tb_xmin_c_global_ycoord,
                        'Global progress tb ymin x coord':tb_ymin_c_global_xcoord,'Global progress tb ymin y coord':tb_ymin_c_global_ycoord}

with open('/data/scratch/aalexopoulos/coords_outputs/global_progress_50_coords.csv','w') as f:
    for key in global_progress_coords.keys():
        f.write("%s, %s\n" % (key, global_progress_coords[key]))

In [10]:
xmin_coords = np.array([[tb_xmin_c_global_xcoord,tb_xmin_c_global_ycoord],[fb_xmin_c_global_xcoord,fb_xmin_c_global_ycoord]])
xmin_coords_idx = np.where(xmin_coords[:,0]==np.amin(xmin_coords[:,0]))
rect_xmin_global = xmin_coords[xmin_coords_idx][0,0]

ymin_coords_idx = np.where(xmin_coords[:,1]==np.amin(xmin_coords[:,1]))
rect_ymin_global = xmin_coords[ymin_coords_idx][0,1]

xmax_coords = np.array([[tb_xmax_c_global_xcoord,tb_xmax_c_global_ycoord],[fb_xmax_c_global_xcoord,fb_xmax_c_global_ycoord]])
xmax_coords_idx = np.where(xmax_coords[:,0]==np.amax(xmax_coords[:,0]))
rect_xmax_global = xmax_coords[xmax_coords_idx][0,0]

ymax_coords_idx = np.where(xmax_coords[:,1]==np.amax(xmax_coords[:,1]))
rect_ymax_global = xmax_coords[ymax_coords_idx][0,1]

print('RECTANGULAR COORDS FOR progress GROUP')
print(xmin_coords)
print('Rect xmin coord: {}, ymin coord: {}'.format(rect_xmin_global, rect_ymin_global))
print(xmax_coords)
print('Rect xmax coord: {}, ymax coord: {}'.format(rect_xmax_global, rect_ymax_global))

rect_width = rect_xmax_global-rect_xmin_global
rect_height = rect_ymax_global-rect_ymin_global

print('Rect Width: {}, Rect Height: {}'.format(rect_width,rect_height))

RECTANGULAR COORDS FOR CONTROL GROUP
[[ 43 250]
 [ 33 135]]
Rect xmin coord: 33, ymin coord: 135
[[303 271]
 [319 149]]
Rect xmax coord: 319, ymax coord: 271
Rect Width: 286, Rect Height: 136


In [11]:
rect_progress_coords = {'progress rect xmin coord': rect_xmin_global,'progress rect ymin coord': rect_ymin_global, 
                        'progress rect xmax coord': rect_xmax_global,'progress rect ymax coord': rect_ymax_global,
                        'progress rect width': rect_width,'progress rect height': rect_height }
with open('/media/anastasis/My Harddisk2/EMC_Thesis/ImageCropping/coords_outputs/rect_progress_50_coords.csv','w') as f:
    for key in rect_progress_coords.keys():
        f.write("%s, %s\n" % (key, rect_progress_coords[key]))

### ---------------- HERE progress GROUP 50 - 100 ---------------- ### 

In [5]:
# HERE WE CREATE THE LISTS THAT WILL HOLD THE COORDINDATES OF MIN AND MAX TIBIAL AND FEMURAL VALUES
progress_femural_edges_50_100 = []
progress_tibial_edges_50_100 = []

progress_fb_xmin_c_50_100 = []
progress_fb_xmax_c_50_100 = []
progress_tb_xmin_c_50_100 = []
progress_tb_xmax_c_50_100 = []
progress_tb_ymin_c_50_100 = []

# bla = ['p_num','FB Xmin Xcoord','FB Xmin Ycoord','FB Xmax Xcoord', 'FB Xmax Ycoord',
        # 'TB Xmin Xcoord','TB Xmin Ycoord','TB Xmax Xcoord', 'TB Xmax Ycoord','TB Ymin Xcoord','TB Ymin Ycoord']

progress_all_coords_50_100 = np.empty((0,11),dtype=int)

In [6]:
k = 0

for i in range(50,len(progress_imgs[:100])):
    mask = sitk.ReadImage(progress_mask[i])
    mask_np = sitk.GetArrayFromImage(mask)
    
    image = sitk.ReadImage(progress_imgs[i])
    image_np = sitk.GetArrayFromImage(image)

    knee_edges, fb_edges, tb_edges = edges_calc(mask_np)
    progress_femural_edges_50_100.append(fb_edges)
    progress_tibial_edges_50_100.append(tb_edges)
    
    fb_xmin_c, fb_xmax_c, tb_xmin_c, tb_xmax_c, tb_ymin_c = max_min_coords(fb_edges, tb_edges)
    progress_fb_xmin_c_50_100.append(fb_xmin_c)
    progress_fb_xmax_c_50_100.append(fb_xmax_c)
    progress_tb_xmin_c_50_100.append(tb_xmin_c)
    progress_tb_xmax_c_50_100.append(tb_xmax_c)
    progress_tb_ymin_c_50_100.append(tb_ymin_c)

    all_info = np.array([[k,fb_xmin_c[0],fb_xmin_c[1],fb_xmax_c[0],fb_xmax_c[1],tb_xmin_c[0],tb_xmin_c[1],tb_xmax_c[0],tb_xmax_c[1],tb_ymin_c[0],tb_ymin_c[1]]])
    progress_all_coords_50_100 = np.append(progress_all_coords_50_100,all_info,axis=0)
    
    print(progress_imgs[i].split('/')[-1],k)
    k = k +1



9131496_IW-TSE_R.nii.gz
9133464_IW-TSE_R.nii.gz
9137556_IW-TSE_R.nii.gz
9141244_IW-TSE_R.nii.gz
9143628_IW-TSE_R.nii.gz
9147692_IW-TSE_R.nii.gz
9149679_IW-TSE_L.nii.gz
9150875_IW-TSE_L.nii.gz
9150875_IW-TSE_R.nii.gz
9153013_IW-TSE_R.nii.gz
9153255_IW-TSE_R.nii.gz
9153696_IW-TSE_L.nii.gz
9153792_IW-TSE_L.nii.gz
9154257_IW-TSE_R.nii.gz
9159961_IW-TSE_R.nii.gz
9160401_IW-TSE_R.nii.gz
9161024_IW-TSE_L.nii.gz
9168012_IW-TSE_L.nii.gz
9176086_IW-TSE_L.nii.gz
9176200_IW-TSE_R.nii.gz
9176441_IW-TSE_R.nii.gz
9176992_IW-TSE_L.nii.gz
9178780_IW-TSE_L.nii.gz
9181583_IW-TSE_L.nii.gz
9185786_IW-TSE_R.nii.gz
9187595_IW-TSE_L.nii.gz
9189553_IW-TSE_L.nii.gz
9189553_IW-TSE_R.nii.gz
9191533_IW-TSE_R.nii.gz
9197064_IW-TSE_L.nii.gz
9198531_IW-TSE_L.nii.gz
9199024_IW-TSE_R.nii.gz
9202047_IW-TSE_R.nii.gz
9208283_IW-TSE_L.nii.gz
9208283_IW-TSE_R.nii.gz
9210505_IW-TSE_R.nii.gz
9214480_IW-TSE_R.nii.gz
9216735_IW-TSE_R.nii.gz
9217260_IW-TSE_L.nii.gz
9217911_IW-TSE_R.nii.gz
9219596_IW-TSE_R.nii.gz
9220251_IW-TSE_R

In [7]:
progress_fb_xmax_c_50_100_np = np.array(progress_fb_xmax_c_50_100)
progress_fb_xmin_c_50_100_np = np.array(progress_fb_xmin_c_50_100)
progress_tb_xmax_c_50_100_np = np.array(progress_tb_xmax_c_50_100)
progress_tb_xmin_c_50_100_np = np.array(progress_tb_xmin_c_50_100)
progress_tb_ymin_c_50_100_np = np.array(progress_tb_ymin_c_50_100)

In [ ]:
save_path_50_100 = '/data/scratch/aalexopoulos/coords_outputs/all_progress_50_100_coords.csv'
np.savetxt(save_path_50_100,progress_all_coords_50_100,delimiter=',')


In [8]:
fb_xmax_c_global = progress_fb_xmax_c_50_100_np[np.where(progress_fb_xmax_c_50_100_np[:,0] == np.amax(progress_fb_xmax_c_50_100_np[:,0]))]
fb_xmax_c_global_xcoord = fb_xmax_c_global[0][0]
fb_xmax_c_global_ycoord = fb_xmax_c_global[0][1]
print('Global progress femural xmax x coord: {}, y coord: {}'.format(fb_xmax_c_global_xcoord, fb_xmax_c_global_ycoord))
idx_fb_xmax_50_100 = 50 + int(np.where(progress_fb_xmax_c_50_100_np[:,0] == np.amax(progress_fb_xmax_c_50_100_np[:,0]))[0])
print(idx_fb_xmax_50_100,progress_imgs[idx_fb_xmax_50_100].split('/')[-1])

fb_xmin_c_global = progress_fb_xmin_c_50_100_np[np.where(progress_fb_xmin_c_50_100_np[:,0] == np.amin(progress_fb_xmin_c_50_100_np[:,0]))]
fb_xmin_c_global_xcoord = fb_xmin_c_global[0][0]
fb_xmin_c_global_ycoord = fb_xmin_c_global[0][1]
print('Global progress femural xmin x coord: {}, y coord: {}'.format(fb_xmin_c_global_xcoord, fb_xmin_c_global_ycoord))
idx_fb_xmin_50_100 = 50 + int(np.where(progress_fb_xmin_c_50_100_np[:,0] == np.amin(progress_fb_xmin_c_50_100_np[:,0]))[0])
print(idx_fb_xmin_50_100,progress_imgs[idx_fb_xmin_50_100].split('/')[-1])

tb_xmax_c_global = progress_tb_xmax_c_50_100_np[np.where(progress_tb_xmax_c_50_100_np[:,0] == np.amax(progress_tb_xmax_c_50_100_np[:,0]))]
tb_xmax_c_global_xcoord = tb_xmax_c_global[0][0]
tb_xmax_c_global_ycoord = tb_xmax_c_global[0][1]
print('Global progress tibial xmax x coord: {}, y coord: {}'.format(tb_xmax_c_global_xcoord, tb_xmax_c_global_ycoord))
idx_tb_xmax_50_100 = 50 + int(np.where(progress_tb_xmax_c_50_100_np[:,0] == np.amax(progress_tb_xmax_c_50_100_np[:,0]))[0])
print(idx_tb_xmax_50_100,progress_imgs[idx_tb_xmax_50_100].split('/')[-1])

tb_xmin_c_global = progress_tb_xmin_c_50_100_np[np.where(progress_tb_xmin_c_50_100_np[:,0] == np.amin(progress_tb_xmin_c_50_100_np[:,0]))]
tb_xmin_c_global_xcoord = tb_xmin_c_global[0][0]
tb_xmin_c_global_ycoord = tb_xmin_c_global[0][1]
print('Global progress tibial xmin x coord: {}, y coord: {}'.format(tb_xmin_c_global_xcoord, tb_xmin_c_global_ycoord))
idx_tb_xmin_50_100 = 50 + int(np.where(progress_tb_xmin_c_50_100_np[:,0] == np.amin(progress_tb_xmin_c_50_100_np[:,0]))[0])
print(idx_tb_xmin_50_100,progress_imgs[idx_tb_xmin_50_100].split('/')[-1])

tb_ymin_c_global = progress_tb_ymin_c_50_100_np[np.where(progress_tb_ymin_c_50_100_np[:,1] == np.amin(progress_tb_ymin_c_50_100_np[:,1]))]
tb_ymin_c_global_xcoord = tb_ymin_c_global[0][0]
tb_ymin_c_global_ycoord = tb_ymin_c_global[0][1]
print('Global progress tibial ymin x coord: {}, y coord: {}'.format(tb_ymin_c_global_xcoord, tb_ymin_c_global_ycoord))
idx_tb_ymin_50_100 = 50 + int(np.where(progress_tb_ymin_c_50_100_np[:,1] == np.amin(progress_tb_ymin_c_50_100_np[:,1]))[0])
print(idx_tb_ymin_50_100,progress_imgs[idx_tb_ymin_50_100].split('/')[-1])
tb_ymin_c_global, tb_xmin_c_global, tb_xmax_c_global, fb_xmin_c_global, fb_xmax_c_global

Global control femural xmax x coord: 331, y coord: 133
91 9220251_IW-TSE_R.nii.gz
Global control femural xmin x coord: 50, y coord: 159
72 9178780_IW-TSE_L.nii.gz
Global control tibial xmax x coord: 310, y coord: 261
91 9220251_IW-TSE_R.nii.gz
Global control tibial xmin x coord: 57, y coord: 267
72 9178780_IW-TSE_L.nii.gz
Global control tibial ymin x coord: 170, y coord: 173
66 9161024_IW-TSE_L.nii.gz


(array([[170, 173]]),
 array([[ 57, 267]]),
 array([[310, 261]]),
 array([[ 50, 159]]),
 array([[331, 133]]))

In [9]:
global_progress_coords = {'Global progress FB xmax x coord':fb_xmax_c_global_xcoord,'Global progress FB xmax y coord':fb_xmax_c_global_ycoord,
                        'Global progress FB xmin x coord':fb_xmin_c_global_xcoord,'Global progress FB xmin y coord':fb_xmin_c_global_ycoord,
                        'Global progress TB xmax x coord':tb_xmax_c_global_xcoord,'Global progress tb xmax y coord':tb_xmax_c_global_ycoord,
                        'Global progress tb xmin x coord':tb_xmin_c_global_xcoord,'Global progress tb xmin y coord':tb_xmin_c_global_ycoord,
                        'Global progress tb ymin x coord':tb_ymin_c_global_xcoord,'Global progress tb ymin y coord':tb_ymin_c_global_ycoord}

with open('/data/scratch/aalexopoulos/coords_outputs/global_progress_50_100_coords.csv','w') as f:
    for key in global_progress_coords.keys():
        f.write("%s, %s\n" % (key, global_progress_coords[key]))

In [10]:
xmin_coords = np.array([[tb_xmin_c_global_xcoord,tb_xmin_c_global_ycoord],[fb_xmin_c_global_xcoord,fb_xmin_c_global_ycoord]])
xmin_coords_idx = np.where(xmin_coords[:,0]==np.amin(xmin_coords[:,0]))
rect_xmin_global = xmin_coords[xmin_coords_idx][0,0]

ymin_coords_idx = np.where(xmin_coords[:,1]==np.amin(xmin_coords[:,1]))
rect_ymin_global = xmin_coords[ymin_coords_idx][0,1]

xmax_coords = np.array([[tb_xmax_c_global_xcoord,tb_xmax_c_global_ycoord],[fb_xmax_c_global_xcoord,fb_xmax_c_global_ycoord]])
xmax_coords_idx = np.where(xmax_coords[:,0]==np.amax(xmax_coords[:,0]))
rect_xmax_global = xmax_coords[xmax_coords_idx][0,0]

ymax_coords_idx = np.where(xmax_coords[:,1]==np.amax(xmax_coords[:,1]))
rect_ymax_global = xmax_coords[ymax_coords_idx][0,1]

print('RECTANGULAR COORDS FOR progress GROUP')
print(xmin_coords)
print('Rect xmin coord: {}, ymin coord: {}'.format(rect_xmin_global, rect_ymin_global))
print(xmax_coords)
print('Rect xmax coord: {}, ymax coord: {}'.format(rect_xmax_global, rect_ymax_global))

rect_width = rect_xmax_global-rect_xmin_global
rect_height = rect_ymax_global-rect_ymin_global

print('Rect Width: {}, Rect Height: {}'.format(rect_width,rect_height))

RECTANGULAR COORDS FOR CONTROL GROUP
[[ 57 267]
 [ 50 159]]
Rect xmin coord: 50, ymin coord: 159
[[310 261]
 [331 133]]
Rect xmax coord: 331, ymax coord: 261
Rect Width: 281, Rect Height: 102


In [11]:
rect_progress_coords = {'progress rect xmin coord': rect_xmin_global,'progress rect ymin coord': rect_ymin_global, 
                        'progress rect xmax coord': rect_xmax_global,'progress rect ymax coord': rect_ymax_global,
                        'progress rect width': rect_width,'progress rect height': rect_height }
with open('/data/scratch/aalexopoulos/coords_outputs/rect_progress_50_100_coords.csv','w') as f:
    for key in rect_progress_coords.keys():
        f.write("%s, %s\n" % (key, rect_progress_coords[key]))

### HERE progress GROUP 100 150 ###

In [12]:
# HERE WE CREATE THE LISTS THAT WILL HOLD THE COORDINDATES OF MIN AND MAX TIBIAL AND FEMURAL VALUES
progress_femural_edges_100_150 = []
progress_tibial_edges_100_150 = []

progress_fb_xmin_c_100_150 = []
progress_fb_xmax_c_100_150 = []
progress_tb_xmin_c_100_150 = []
progress_tb_xmax_c_100_150 = []
progress_tb_ymin_c_100_150 = []

# bla = ['p_num','FB Xmin Xcoord','FB Xmin Ycoord','FB Xmax Xcoord', 'FB Xmax Ycoord',
        # 'TB Xmin Xcoord','TB Xmin Ycoord','TB Xmax Xcoord', 'TB Xmax Ycoord','TB Ymin Xcoord','TB Ymin Ycoord']

progress_all_coords_100_150 = np.empty((0,11),dtype=int)

In [13]:
k = 0

for i in range(100,len(progress_imgs[:150])):
    mask = sitk.ReadImage(progress_mask[i])
    mask_np = sitk.GetArrayFromImage(mask)
    
    image = sitk.ReadImage(progress_imgs[i])
    image_np = sitk.GetArrayFromImage(image)

    knee_edges, fb_edges, tb_edges = edges_calc(mask_np)
    progress_femural_edges_100_150.append(fb_edges)
    progress_tibial_edges_100_150.append(tb_edges)
    
    fb_xmin_c, fb_xmax_c, tb_xmin_c, tb_xmax_c, tb_ymin_c = max_min_coords(fb_edges, tb_edges)
    progress_fb_xmin_c_100_150.append(fb_xmin_c)
    progress_fb_xmax_c_100_150.append(fb_xmax_c)
    progress_tb_xmin_c_100_150.append(tb_xmin_c)
    progress_tb_xmax_c_100_150.append(tb_xmax_c)
    progress_tb_ymin_c_100_150.append(tb_ymin_c)

    all_info = np.array([[k,fb_xmin_c[0],fb_xmin_c[1],fb_xmax_c[0],fb_xmax_c[1],tb_xmin_c[0],tb_xmin_c[1],tb_xmax_c[0],tb_xmax_c[1],tb_ymin_c[0],tb_ymin_c[1]]])
    progress_all_coords_100_150 = np.append(progress_all_coords_100_150,all_info,axis=0)
    
    print(progress_imgs[i].split('/')[-1],k)
    k = k +1



9242547_IW-TSE_R.nii.gz
9245448_IW-TSE_R.nii.gz
9246995_IW-TSE_L.nii.gz
9247270_IW-TSE_L.nii.gz
9247270_IW-TSE_R.nii.gz
9248710_IW-TSE_R.nii.gz
9251736_IW-TSE_L.nii.gz
9252019_IW-TSE_R.nii.gz
9252576_IW-TSE_R.nii.gz
9252629_IW-TSE_L.nii.gz
9253390_IW-TSE_L.nii.gz
9258864_IW-TSE_L.nii.gz
9258864_IW-TSE_R.nii.gz
9264046_IW-TSE_L.nii.gz
9268052_IW-TSE_L.nii.gz
9268509_IW-TSE_R.nii.gz
9268652_IW-TSE_R.nii.gz
9268724_IW-TSE_R.nii.gz
9270566_IW-TSE_R.nii.gz
9272198_IW-TSE_L.nii.gz
9273128_IW-TSE_L.nii.gz
9275258_IW-TSE_L.nii.gz
9275258_IW-TSE_R.nii.gz
9275309_IW-TSE_R.nii.gz
9282182_IW-TSE_R.nii.gz
9285207_IW-TSE_R.nii.gz
9285212_IW-TSE_R.nii.gz
9288025_IW-TSE_R.nii.gz
9290697_IW-TSE_L.nii.gz
9291210_IW-TSE_R.nii.gz
9292081_IW-TSE_L.nii.gz
9301641_IW-TSE_L.nii.gz
9303044_IW-TSE_L.nii.gz
9305336_IW-TSE_R.nii.gz
9312419_IW-TSE_L.nii.gz
9314056_IW-TSE_R.nii.gz
9314340_IW-TSE_R.nii.gz
9315555_IW-TSE_L.nii.gz
9316108_IW-TSE_L.nii.gz
9316905_IW-TSE_L.nii.gz
9321380_IW-TSE_L.nii.gz
9322401_IW-TSE_R

In [14]:
progress_fb_xmax_c_100_150_np = np.array(progress_fb_xmax_c_100_150)
progress_fb_xmin_c_100_150_np = np.array(progress_fb_xmin_c_100_150)
progress_tb_xmax_c_100_150_np = np.array(progress_tb_xmax_c_100_150)
progress_tb_xmin_c_100_150_np = np.array(progress_tb_xmin_c_100_150)
progress_tb_ymin_c_100_150_np = np.array(progress_tb_ymin_c_100_150)

In [ ]:
save_path_100_150 = '/data/scratch/aalexopoulos/coords_outputs/all_progress_100_150_coords.csv'
np.savetxt(save_path_100_150,progress_all_coords_100_150,delimiter=',')


In [15]:
fb_xmax_c_global = progress_fb_xmax_c_100_150_np[np.where(progress_fb_xmax_c_100_150_np[:,0] == np.amax(progress_fb_xmax_c_100_150_np[:,0]))]
fb_xmax_c_global_xcoord = fb_xmax_c_global[0][0]
fb_xmax_c_global_ycoord = fb_xmax_c_global[0][1]
print('Global progress femural xmax x coord: {}, y coord: {}'.format(fb_xmax_c_global_xcoord, fb_xmax_c_global_ycoord))
idx_fb_xmax_100_150 = 100 + int(np.where(progress_fb_xmax_c_100_150_np[:,0] == np.amax(progress_fb_xmax_c_100_150_np[:,0]))[0])
print(idx_fb_xmax_100_150,progress_imgs[idx_fb_xmax_100_150].split('/')[-1])

fb_xmin_c_global = progress_fb_xmin_c_100_150_np[np.where(progress_fb_xmin_c_100_150_np[:,0] == np.amin(progress_fb_xmin_c_100_150_np[:,0]))]
fb_xmin_c_global_xcoord = fb_xmin_c_global[0][0]
fb_xmin_c_global_ycoord = fb_xmin_c_global[0][1]
print('Global progress femural xmin x coord: {}, y coord: {}'.format(fb_xmin_c_global_xcoord, fb_xmin_c_global_ycoord))
idx_fb_xmin_100_150 = 100 + int(np.where(progress_fb_xmin_c_100_150_np[:,0] == np.amin(progress_fb_xmin_c_100_150_np[:,0]))[0])
print(idx_fb_xmin_100_150,progress_imgs[idx_fb_xmin_100_150].split('/')[-1])

tb_xmax_c_global = progress_tb_xmax_c_100_150_np[np.where(progress_tb_xmax_c_100_150_np[:,0] == np.amax(progress_tb_xmax_c_100_150_np[:,0]))]
tb_xmax_c_global_xcoord = tb_xmax_c_global[0][0]
tb_xmax_c_global_ycoord = tb_xmax_c_global[0][1]
print('Global progress tibial xmax x coord: {}, y coord: {}'.format(tb_xmax_c_global_xcoord, tb_xmax_c_global_ycoord))
idx_tb_xmax_100_150 = 100 + int(np.where(progress_tb_xmax_c_100_150_np[:,0] == np.amax(progress_tb_xmax_c_100_150_np[:,0]))[0])
print(idx_tb_xmax_100_150,progress_imgs[idx_tb_xmax_100_150].split('/')[-1])

tb_xmin_c_global = progress_tb_xmin_c_100_150_np[np.where(progress_tb_xmin_c_100_150_np[:,0] == np.amin(progress_tb_xmin_c_100_150_np[:,0]))]
tb_xmin_c_global_xcoord = tb_xmin_c_global[0][0]
tb_xmin_c_global_ycoord = tb_xmin_c_global[0][1]
print('Global progress tibial xmin x coord: {}, y coord: {}'.format(tb_xmin_c_global_xcoord, tb_xmin_c_global_ycoord))
idx_tb_xmin_100_150 = 100 + int(np.where(progress_tb_xmin_c_100_150_np[:,0] == np.amin(progress_tb_xmin_c_100_150_np[:,0]))[0])
print(idx_tb_xmin_100_150,progress_imgs[idx_tb_xmin_100_150].split('/')[-1])

tb_ymin_c_global = progress_tb_ymin_c_100_150_np[np.where(progress_tb_ymin_c_100_150_np[:,1] == np.amin(progress_tb_ymin_c_100_150_np[:,1]))]
tb_ymin_c_global_xcoord = tb_ymin_c_global[0][0]
tb_ymin_c_global_ycoord = tb_ymin_c_global[0][1]
print('Global progress tibial ymin x coord: {}, y coord: {}'.format(tb_ymin_c_global_xcoord, tb_ymin_c_global_ycoord))
idx_tb_ymin_100_150 = 100 + int(np.where(progress_tb_ymin_c_100_150_np[:,1] == np.amin(progress_tb_ymin_c_100_150_np[:,1]))[0])
print(idx_tb_ymin_100_150,progress_imgs[idx_tb_ymin_100_150].split('/')[-1])
tb_ymin_c_global, tb_xmin_c_global, tb_xmax_c_global, fb_xmin_c_global, fb_xmax_c_global

Global control femural xmax x coord: 333, y coord: 157
141 9322401_IW-TSE_R.nii.gz
Global control femural xmin x coord: 43, y coord: 158
133 9305336_IW-TSE_R.nii.gz
Global control tibial xmax x coord: 313, y coord: 258
115 9268509_IW-TSE_R.nii.gz
Global control tibial xmin x coord: 48, y coord: 255
140 9321380_IW-TSE_L.nii.gz
Global control tibial ymin x coord: 189, y coord: 186
131 9301641_IW-TSE_L.nii.gz


(array([[189, 186]]),
 array([[ 48, 255]]),
 array([[313, 258]]),
 array([[ 43, 158]]),
 array([[333, 157]]))

In [10]:
global_progress_coords = {'Global progress FB xmax x coord':fb_xmax_c_global_xcoord,'Global progress FB xmax y coord':fb_xmax_c_global_ycoord,
                        'Global progress FB xmin x coord':fb_xmin_c_global_xcoord,'Global progress FB xmin y coord':fb_xmin_c_global_ycoord,
                        'Global progress TB xmax x coord':tb_xmax_c_global_xcoord,'Global progress tb xmax y coord':tb_xmax_c_global_ycoord,
                        'Global progress tb xmin x coord':tb_xmin_c_global_xcoord,'Global progress tb xmin y coord':tb_xmin_c_global_ycoord,
                        'Global progress tb ymin x coord':tb_ymin_c_global_xcoord,'Global progress tb ymin y coord':tb_ymin_c_global_ycoord}

with open('/data/scratch/aalexopoulos/coords_outputs/global_progress_100_150_coords.csv','w') as f:
    for key in global_progress_coords.keys():
        f.write("%s, %s\n" % (key, global_progress_coords[key]))

In [11]:
xmin_coords = np.array([[tb_xmin_c_global_xcoord,tb_xmin_c_global_ycoord],[fb_xmin_c_global_xcoord,fb_xmin_c_global_ycoord]])
xmin_coords_idx = np.where(xmin_coords[:,0]==np.amin(xmin_coords[:,0]))
rect_xmin_global = xmin_coords[xmin_coords_idx][0,0]

ymin_coords_idx = np.where(xmin_coords[:,1]==np.amin(xmin_coords[:,1]))
rect_ymin_global = xmin_coords[ymin_coords_idx][0,1]

xmax_coords = np.array([[tb_xmax_c_global_xcoord,tb_xmax_c_global_ycoord],[fb_xmax_c_global_xcoord,fb_xmax_c_global_ycoord]])
xmax_coords_idx = np.where(xmax_coords[:,0]==np.amax(xmax_coords[:,0]))
rect_xmax_global = xmax_coords[xmax_coords_idx][0,0]

ymax_coords_idx = np.where(xmax_coords[:,1]==np.amax(xmax_coords[:,1]))
rect_ymax_global = xmax_coords[ymax_coords_idx][0,1]

print('RECTANGULAR COORDS FOR progress GROUP')
print(xmin_coords)
print('Rect xmin coord: {}, ymin coord: {}'.format(rect_xmin_global, rect_ymin_global))
print(xmax_coords)
print('Rect xmax coord: {}, ymax coord: {}'.format(rect_xmax_global, rect_ymax_global))

rect_width = rect_xmax_global-rect_xmin_global
rect_height = rect_ymax_global-rect_ymin_global

print('Rect Width: {}, Rect Height: {}'.format(rect_width,rect_height))

RECTANGULAR COORDS FOR CONTROL GROUP
[[ 38 296]
 [ 27 264]]
Rect xmin coord: 27, ymin coord: 264
[[313 258]
 [333 157]]
Rect xmax coord: 333, ymax coord: 258
Rect Width: 306, Rect Height: -6


In [12]:
rect_progress_coords = {'progress rect xmin coord': rect_xmin_global,'progress rect ymin coord': rect_ymin_global, 
                        'progress rect xmax coord': rect_xmax_global,'progress rect ymax coord': rect_ymax_global,
                        'progress rect width': rect_width,'progress rect height': rect_height }
with open('/data/scratch/aalexopoulos/coords_outputs/rect_progress_100_150_coords.csv','w') as f:
    for key in rect_progress_coords.keys():
        f.write("%s, %s\n" % (key, rect_progress_coords[key]))

### HERE progress 150-200 ###

In [5]:
# HERE WE CREATE THE LISTS THAT WILL HOLD THE COORDINDATES OF MIN AND MAX TIBIAL AND FEMURAL VALUES
progress_femural_edges_150_200 = []
progress_tibial_edges_150_200 = []

progress_fb_xmin_c_150_200 = []
progress_fb_xmax_c_150_200 = []
progress_tb_xmin_c_150_200 = []
progress_tb_xmax_c_150_200 = []
progress_tb_ymin_c_150_200 = []

# bla = ['p_num','FB Xmin Xcoord','FB Xmin Ycoord','FB Xmax Xcoord', 'FB Xmax Ycoord',
        # 'TB Xmin Xcoord','TB Xmin Ycoord','TB Xmax Xcoord', 'TB Xmax Ycoord','TB Ymin Xcoord','TB Ymin Ycoord']

progress_all_coords_150_200 = np.empty((0,11),dtype=int)


In [6]:
k = 0

for i in range(150,len(progress_imgs)):
    mask = sitk.ReadImage(progress_mask[i])
    mask_np = sitk.GetArrayFromImage(mask)
    
    image = sitk.ReadImage(progress_imgs[i])
    image_np = sitk.GetArrayFromImage(image)

    knee_edges, fb_edges, tb_edges = edges_calc(mask_np)
    progress_femural_edges_150_200.append(fb_edges)
    progress_tibial_edges_150_200.append(tb_edges)

    fb_xmin_c, fb_xmax_c, tb_xmin_c, tb_xmax_c, tb_ymin_c = max_min_coords(fb_edges, tb_edges)

    progress_fb_xmin_c_150_200.append(fb_xmin_c)
    progress_fb_xmax_c_150_200.append(fb_xmax_c)
    progress_tb_xmin_c_150_200.append(tb_xmin_c)
    progress_tb_xmax_c_150_200.append(tb_xmax_c)
    progress_tb_ymin_c_150_200.append(tb_ymin_c)

    all_info = np.array([[k,fb_xmin_c[0],fb_xmin_c[1],fb_xmax_c[0],fb_xmax_c[1],tb_xmin_c[0],tb_xmin_c[1],tb_xmax_c[0],tb_xmax_c[1],tb_ymin_c[0],tb_ymin_c[1]]])
    progress_all_coords_150_200 = np.append(progress_all_coords_150_200,all_info,axis=0)
    
    print(progress_imgs[i].split('/')[-1],k)
    k = k +1


9357383_IW-TSE_R.nii.gz 1
9357990_IW-TSE_L.nii.gz 2
9358131_IW-TSE_R.nii.gz 3
9362264_IW-TSE_L.nii.gz 4
9364232_IW-TSE_L.nii.gz 5
9364232_IW-TSE_R.nii.gz 6
9368408_IW-TSE_R.nii.gz 7
9370170_IW-TSE_R.nii.gz 8
9372219_IW-TSE_R.nii.gz 9
9373065_IW-TSE_R.nii.gz 10
9373786_IW-TSE_R.nii.gz 11
9374235_IW-TSE_R.nii.gz 12
9376430_IW-TSE_R.nii.gz 13
9379540_IW-TSE_R.nii.gz 14
9381081_IW-TSE_R.nii.gz 15
9385984_IW-TSE_L.nii.gz 16
9385984_IW-TSE_R.nii.gz 17
9386991_IW-TSE_R.nii.gz 18
9387130_IW-TSE_L.nii.gz 19
9389941_IW-TSE_R.nii.gz 20
9392241_IW-TSE_L.nii.gz 21
9393127_IW-TSE_L.nii.gz 22
9393171_IW-TSE_R.nii.gz 23
9394136_IW-TSE_R.nii.gz 24
9395235_IW-TSE_L.nii.gz 25
9396054_IW-TSE_R.nii.gz 26
9398272_IW-TSE_R.nii.gz 27
9402139_IW-TSE_R.nii.gz 28
9403165_IW-TSE_R.nii.gz 29
9408506_IW-TSE_L.nii.gz 30
9409969_IW-TSE_L.nii.gz 31
9410036_IW-TSE_R.nii.gz 32
9410941_IW-TSE_R.nii.gz 33
9412037_IW-TSE_L.nii.gz 34
9412037_IW-TSE_R.nii.gz 35
9414968_IW-TSE_L.nii.gz 36
9416381_IW-TSE_L.nii.gz 37
9417063_IW

In [7]:
progress_fb_xmax_c_150_200_np = np.array(progress_fb_xmax_c_150_200)
progress_fb_xmin_c_150_200_np = np.array(progress_fb_xmin_c_150_200)
progress_tb_xmax_c_150_200_np = np.array(progress_tb_xmax_c_150_200)
progress_tb_xmin_c_150_200_np = np.array(progress_tb_xmin_c_150_200)
progress_tb_ymin_c_150_200_np = np.array(progress_tb_ymin_c_150_200)

In [ ]:
save_path_150_200 = '/data/scratch/aalexopoulos/coords_outputs/all_progress_150_200_coords.csv'
np.savetxt(save_path_150_200,progress_all_coords_150_200,delimiter=',')


In [8]:
fb_xmax_c_global = progress_fb_xmax_c_150_200_np[np.where(progress_fb_xmax_c_150_200_np[:,0] == np.amax(progress_fb_xmax_c_150_200_np[:,0]))]
fb_xmax_c_global_xcoord = fb_xmax_c_global[0][0]
fb_xmax_c_global_ycoord = fb_xmax_c_global[0][1]
print('Global progress femural xmax x coord: {}, y coord: {}'.format(fb_xmax_c_global_xcoord, fb_xmax_c_global_ycoord))
idx_fb_xmax_150_200 = 150 + int(np.where(progress_fb_xmax_c_150_200_np[:,0] == np.amax(progress_fb_xmax_c_150_200_np[:,0]))[0])
print(idx_fb_xmax_150_200,progress_imgs[idx_fb_xmax_150_200].split('/')[-1])

fb_xmin_c_global = progress_fb_xmin_c_150_200_np[np.where(progress_fb_xmin_c_150_200_np[:,0] == np.amin(progress_fb_xmin_c_150_200_np[:,0]))]
fb_xmin_c_global_xcoord = fb_xmin_c_global[0][0]
fb_xmin_c_global_ycoord = fb_xmin_c_global[0][1]
print('Global progress femural xmin x coord: {}, y coord: {}'.format(fb_xmin_c_global_xcoord, fb_xmin_c_global_ycoord))
idx_fb_xmin_150_200 = 150 + int(np.where(progress_fb_xmin_c_150_200_np[:,0] == np.amin(progress_fb_xmin_c_150_200_np[:,0]))[0])
print(idx_fb_xmin_150_200,progress_imgs[idx_fb_xmin_150_200].split('/')[-1])

tb_xmax_c_global = progress_tb_xmax_c_150_200_np[np.where(progress_tb_xmax_c_150_200_np[:,0] == np.amax(progress_tb_xmax_c_150_200_np[:,0]))]
tb_xmax_c_global_xcoord = tb_xmax_c_global[0][0]
tb_xmax_c_global_ycoord = tb_xmax_c_global[0][1]
print('Global progress tibial xmax x coord: {}, y coord: {}'.format(tb_xmax_c_global_xcoord, tb_xmax_c_global_ycoord))
idx_tb_xmax_150_200 = 150 + int(np.where(progress_tb_xmax_c_150_200_np[:,0] == np.amax(progress_tb_xmax_c_150_200_np[:,0]))[0])
print(idx_tb_xmax_150_200,progress_imgs[idx_tb_xmax_150_200].split('/')[-1])

tb_xmin_c_global = progress_tb_xmin_c_150_200_np[np.where(progress_tb_xmin_c_150_200_np[:,0] == np.amin(progress_tb_xmin_c_150_200_np[:,0]))]
tb_xmin_c_global_xcoord = tb_xmin_c_global[0][0]
tb_xmin_c_global_ycoord = tb_xmin_c_global[0][1]
print('Global progress tibial xmin x coord: {}, y coord: {}'.format(tb_xmin_c_global_xcoord, tb_xmin_c_global_ycoord))
idx_tb_xmin_150_200 = 150 + int(np.where(progress_tb_xmin_c_150_200_np[:,0] == np.amin(progress_tb_xmin_c_150_200_np[:,0]))[0])
print(idx_tb_xmin_150_200,progress_imgs[idx_tb_xmin_150_200].split('/')[-1])

tb_ymin_c_global = progress_tb_ymin_c_150_200_np[np.where(progress_tb_ymin_c_150_200_np[:,1] == np.amin(progress_tb_ymin_c_150_200_np[:,1]))]
tb_ymin_c_global_xcoord = tb_ymin_c_global[0][0]
tb_ymin_c_global_ycoord = tb_ymin_c_global[0][1]
print('Global progress tibial ymin x coord: {}, y coord: {}'.format(tb_ymin_c_global_xcoord, tb_ymin_c_global_ycoord))
idx_tb_ymin_150_200 = 150 + int(np.where(progress_tb_ymin_c_150_200_np[:,1] == np.amin(progress_tb_ymin_c_150_200_np[:,1]))[0])
print(idx_tb_ymin_150_200,progress_imgs[idx_tb_ymin_150_200].split('/')[-1])
tb_ymin_c_global, tb_xmin_c_global, tb_xmax_c_global, fb_xmin_c_global, fb_xmax_c_global

Global control femural xmax x coord: 325, y coord: 147
189 9418547_IW-TSE_R.nii.gz
Global control femural xmin x coord: 49, y coord: 147
185 9414968_IW-TSE_L.nii.gz
Global control tibial xmax x coord: 320, y coord: 265
189 9418547_IW-TSE_R.nii.gz
Global control tibial xmin x coord: 53, y coord: 294
185 9414968_IW-TSE_L.nii.gz
Global control tibial ymin x coord: 169, y coord: 182
194 9429147_IW-TSE_R.nii.gz


(array([[169, 182]]),
 array([[ 53, 294]]),
 array([[320, 265]]),
 array([[ 49, 147]]),
 array([[325, 147]]))

In [18]:
global_progress_coords = {'Global progress FB xmax x coord':fb_xmax_c_global_xcoord,'Global progress FB xmax y coord':fb_xmax_c_global_ycoord,
                        'Global progress FB xmin x coord':fb_xmin_c_global_xcoord,'Global progress FB xmin y coord':fb_xmin_c_global_ycoord,
                        'Global progress TB xmax x coord':tb_xmax_c_global_xcoord,'Global progress tb xmax y coord':tb_xmax_c_global_ycoord,
                        'Global progress tb xmin x coord':tb_xmin_c_global_xcoord,'Global progress tb xmin y coord':tb_xmin_c_global_ycoord,
                        'Global progress tb ymin x coord':tb_ymin_c_global_xcoord,'Global progress tb ymin y coord':tb_ymin_c_global_ycoord}

with open('/media/anastasis/My Harddisk2/EMC_Thesis/ImageCropping/coords_outputs/global_progress_150_200_coords.csv','w') as f:
    for key in global_progress_coords.keys():
        f.write("%s, %s\n" % (key, global_progress_coords[key]))

In [9]:
xmin_coords = np.array([[tb_xmin_c_global_xcoord,tb_xmin_c_global_ycoord],[fb_xmin_c_global_xcoord,fb_xmin_c_global_ycoord]])
xmin_coords_idx = np.where(xmin_coords[:,0]==np.amin(xmin_coords[:,0]))
rect_xmin_global = xmin_coords[xmin_coords_idx][0,0]

ymin_coords_idx = np.where(xmin_coords[:,1]==np.amin(xmin_coords[:,1]))
rect_ymin_global = xmin_coords[ymin_coords_idx][0,1]

xmax_coords = np.array([[tb_xmax_c_global_xcoord,tb_xmax_c_global_ycoord],[fb_xmax_c_global_xcoord,fb_xmax_c_global_ycoord]])
xmax_coords_idx = np.where(xmax_coords[:,0]==np.amax(xmax_coords[:,0]))
rect_xmax_global = xmax_coords[xmax_coords_idx][0,0]

ymax_coords_idx = np.where(xmax_coords[:,1]==np.amax(xmax_coords[:,1]))
rect_ymax_global = xmax_coords[ymax_coords_idx][0,1]

print('RECTANGULAR COORDS FOR progress GROUP')
print(xmin_coords)
print('Rect xmin coord: {}, ymin coord: {}'.format(rect_xmin_global, rect_ymin_global))
print(xmax_coords)
print('Rect xmax coord: {}, ymax coord: {}'.format(rect_xmax_global, rect_ymax_global))

rect_width = rect_xmax_global-rect_xmin_global
rect_height = rect_ymax_global-rect_ymin_global

print('Rect Width: {}, Rect Height: {}'.format(rect_width,rect_height))

RECTANGULAR COORDS FOR CONTROL GROUP
[[ 53 294]
 [ 49 147]]
Rect xmin coord: 49, ymin coord: 147
[[320 265]
 [325 147]]
Rect xmax coord: 325, ymax coord: 265
Rect Width: 276, Rect Height: 118


In [10]:
rect_progress_coords = {'progress rect xmin coord': rect_xmin_global,'progress rect ymin coord': rect_ymin_global, 
                        'progress rect xmax coord': rect_xmax_global,'progress rect ymax coord': rect_ymax_global,
                        'progress rect width': rect_width,'progress rect height': rect_height }
with open('/media/anastasis/My Harddisk2/EMC_Thesis/ImageCropping/coords_outputs/rect_progress_150_200_coords.csv','w') as f:
    for key in rect_progress_coords.keys():
        f.write("%s, %s\n" % (key, rect_progress_coords[key]))